# Welcome to Exkaldi

In this section, we will train the triphone HMM-GMM with delta feature.

In [ ]:
import exkaldi

import os
dataDir = "librispeech_dummy"

Firstly, instantiate a triphone model object.

In [ ]:
model = exkaldi.hmm.TriphoneHMM()

model

___model___ is unavaliable now. We have to initialize its data. We will use these files which have been generated in early steps.

___tree___ and ___treeStats___ : generated in 6_train_decision_tree  
___topo___:  generated in 5_train_mono_HMM-GMM

Actually, you can initialize the TriphoneHMM object from feature directly, but here we use tree statistics file.

In [ ]:
treeFile = os.path.join(dataDir, "exp", "train_delta", "tree")
treeStatsFile = os.path.join(dataDir, "exp", "train_delta", "treeStats.acc")
topoFile = os.path.join(dataDir, "exp", "topo")

In [ ]:
model.initialize(tree=treeFile, treeStatsFile=treeStatsFile, topoFile=topoFile)

model.info

The training steps of triphone HMM are almost the same as monophone HMM except that we don't use equally aligning at the first time.

We will introduce the traing step in a nutshell.

### Training in detail

At the first step, we must generate the new alignment data in the first step. You can convert the lastest alignment data generated by monophone model to a new alignment data corresponding to triphone. Use the generated alignment and final monophone model.

In [ ]:
aliFile = os.path.join(dataDir, "exp", "train_mono", "final.ali")
monoFile= os.path.join(dataDir, "exp", "train_mono", "final.mdl")
ali = exkaldi.load_ali(aliFile)

In [ ]:
newAli = exkaldi.hmm.convert_alignment(
                                 ali=ali, 
                                 originHmm=monoFile, 
                                 targetHmm=model, 
                                 tree=treeFile
                            )
newAli

In another way, align feature again directly with new triphone model. 

In the next steps, we will review the steps to train a HMM-GMM model and use it to align acoustic feature.

In [ ]:
del newAli

Prepare a lexicons (generated in 3_prepare_lexicons).

In [ ]:
lexFile = os.path.join(dataDir, "exp", "lexicons.lex")

lexicons = exkaldi.load_lex(lexFile)

lexicons

Prepare int-format transcription (generated in 5_train_mono_HMM-GMM ).

In [ ]:
intTransFile = os.path.join(dataDir, "exp", "text.int")

trans = exkaldi.load_transcription(intTransFile)

trans.subset(nHead=1)

Prepare L.fst file (generated in 3_prepare_lexicons).

In [ ]:
Lfile = os.path.join(dataDir, "exp", "L.fst")

Prepare feature (generated in 2_feature_processing).

In [ ]:
featFile = os.path.join(dataDir, "exp", "train_mfcc_cmvn.ark")
feat = exkaldi.load_feat(featFile)
feat = feat.add_delta(order=2)

feat.dim

#### 1. Compile new train graph.

In [ ]:
outDir = os.path.join(dataDir, "exp", "train_delta")

exkaldi.utils.make_dependent_dirs(outDir, pathIsFile=False)

In [ ]:
trainGraphFile = os.path.join(outDir, "train_graph")

model.compile_train_graph(tree=treeFile, transcription=trans, LFile=Lfile, outFile=trainGraphFile, lexicons=lexicons)

#### 2. Align acoustic feature.

In [ ]:
ali = model.align(feat, trainGraphFile, lexicons=lexicons)

ali

#### 3. Accumulate statistics.

In [ ]:
statsFile = os.path.join(outDir, "stats.acc")

model.accumulate_stats(feat=feat, ali=ali, outFile=statsFile)

#### 4. Update HMM-GMM parameters.

In [ ]:
targetGaussians = 300

model.update(statsFile, targetGaussians)

model.info

### Training in high-level

In this step, we will introduce how to training the triphone in directly.

In [ ]:
del model
del ali
del trans

os.remove(trainGraphFile)
os.remove(statsFile)

Some file paths or objects defined above will be used here. 

Firstly, initialize model. Give lexicons as a optional parameter.

In [ ]:
model = exkaldi.hmm.TriphoneHMM(lexicons=lexicons)

model.initialize(tree=treeFile, treeStatsFile=treeStatsFile, topoFile=topoFile)

model.info

Then train it.

In [ ]:
outDir = os.path.join(dataDir, "exp", "train_delta")
transFile = os.path.join(dataDir, "train", "text")

aliIndex = model.train(feat=feat, transcription=transFile, LFile=Lfile, tree=treeFile, tempDir=outDir, 
                         numIters=10, maxIterInc=8, totgauss=1500
                    )

Final model and alignment have been saved in file automatically. Look the final model information.

In [ ]:
model.info